# ResNet-Competition

__Author:Marcus Alsterman<br>
Name here [marals@kth.se](mailto:marals@kth.se)__

In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random as rn
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255, X_test/255

In [ ]:
# # seeding
# seed = 95
# np.random.seed(seed)
# rn.seed(seed + 1)
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
#                               inter_op_parallelism_threads=1)
# from keras import backend as K
# tf.set_random_seed(seed + 2)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

In [ ]:
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, ReLU, Add, Flatten
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical
ES = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

In [ ]:
# data preprocessing
cutoff = 0.051
mean_train_image = np.mean(X_train, axis=0)
mean_train_image_cutoff = mean_train_image > cutoff
plt.imshow(np.concatenate((mean_train_image ,mean_train_image_cutoff), axis=1), clim=[0,1], cmap='bone')

pixel_count = np.sum(mean_train_image_cutoff)
print('pixel_count = ',pixel_count)

X_train = X_train[:, mean_train_image_cutoff]
X_test = X_test[:, mean_train_image_cutoff]

# one hot
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## MLP

In [ ]:
x_in = Input(shape=(pixel_count,))
x = x_in
num_layers = 5
for i in range(0,num_layers):
    x = Dense(20, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=[x_in], outputs= x)

In [ ]:
model.summary();

In [ ]:
model.compile(optimizer='AdaGrad',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50,
          validation_split=0.1,
          callbacks=[ES],
          shuffle=True)

In [ ]:
print('final accuracy MLP = ',model.evaluate(X_test, y_test)[1])

## ResNet

In [ ]:
def dense_with_skip(x, dense):
    skip = x
    x = dense(x)
    x = Add()([x,skip])
    return x

In [ ]:
def dense_factory(size=19):
    return Dense(size,
                 activation = 'relu',
                 kernel_initializer = initializers.RandomNormal(0.0, 0.01, seed=123),
                 bias_initializer = 'zero',
                 
                
                )
x_in = Input(shape=(pixel_count,))

x = dense_factory()(x_in)

num_layers_with_skip = 7
for i in range(0,num_layers_with_skip):
    x = dense_with_skip(x, dense_factory())

# output layer
x = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=[x_in], outputs= x)

In [ ]:
model.summary();

In [ ]:
model.compile(optimizer='AdaGrad',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50,
          validation_split=0.1,
          callbacks=[ES],
          shuffle=True)

In [ ]:
print('final accuracy ResNet = ',model.evaluate(X_test, y_test)[1])